In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


# Module

In [1]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

### Dataset

In [2]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [3]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [4]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [5]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [7]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [22]:
%%time 

model.fit(x_train_reshaped, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 46us/sample - loss: 0.0126 - accuracy: 0.9964
Epoch 2/10
60000/60000 [==============================] - 3s 51us/sample - loss: 0.0122 - accuracy: 0.9959
Epoch 3/10
60000/60000 [==============================] - 3s 44us/sample - loss: 0.0102 - accuracy: 0.9967
Epoch 4/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.0082 - accuracy: 0.9974
Epoch 5/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.0086 - accuracy: 0.9969
Epoch 6/10
60000/60000 [==============================] - 3s 46us/sample - loss: 0.0075 - accuracy: 0.9974
Epoch 7/10
60000/60000 [==============================] - 3s 46us/sample - loss: 0.0076 - accuracy: 0.9975
Epoch 8/10
60000/60000 [==============================] - 3s 46us/sample - loss: 0.0067 - accuracy: 0.9979
Epoch 9/10
60000/60000 [==============================] - 3s 46us/sample - loss: 0.0053 - accuracy: 0.9983
Epoch 10/10
60

# Model Evaluation

In [30]:
predictions = model.predict(x_test_reshaped)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy*100))

Accuracy on benign test example: 99.05000000000001%


# Adversarial Examples

In [31]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=True)

## 1-1) FGSM_origin
### 원본 이미지 x_test

In [32]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_reshaped,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.2118821144104004
CPU times: user 1.59 s, sys: 75.9 ms, total: 1.67 s
Wall time: 1.21 s


In [33]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 82.07%


## 1-2) FGSM_noise
### noise image -> x_test_noise

In [34]:
# noise 
noise = np.random.normal(0,0.01,x_test.shape)
x_test_noise1 = x_test_norm + noise
x_test_noise1_reshape = x_test_noise1.reshape(-1,28,28,1)

In [35]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise1_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.2127604484558105
CPU times: user 1.57 s, sys: 98.7 ms, total: 1.67 s
Wall time: 1.21 s


In [36]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 81.64%


## noise 2 - sigma-0.03

In [24]:
# noise 2
noise2 = np.random.normal(0,0.03,x_test.shape)
x_test_noise2 = x_test_norm + noise2
x_test_noise2_reshape = x_test_noise2.reshape(-1,28,28,1)

In [25]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise2_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0603363513946533
CPU times: user 1.28 s, sys: 148 ms, total: 1.43 s
Wall time: 1.06 s


In [26]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 80.28%


## noise 3 - sigma-0.05

In [38]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [45]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise3_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0697588920593262
CPU times: user 1.33 s, sys: 116 ms, total: 1.44 s
Wall time: 1.07 s


In [46]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 79.32000000000001%


## noise 4 - sigma-0.07

In [48]:
# noise 4
noise4 = np.random.normal(0,0.07,x_test.shape)
x_test_noise4 = x_test_norm + noise4
x_test_noise4_reshape = x_test_noise4.reshape(-1,28,28,1)

In [49]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise4_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.3033349514007568
CPU times: user 1.66 s, sys: 111 ms, total: 1.77 s
Wall time: 1.3 s


In [50]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 77.52%


## noise 5 - sigma-0.1

In [51]:
# noise 5
noise5 = np.random.normal(0,0.1,x_test.shape)
x_test_noise5 = x_test_norm + noise5
x_test_noise5_reshape = x_test_noise5.reshape(-1,28,28,1)

In [52]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise5_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.190345287322998
CPU times: user 1.47 s, sys: 129 ms, total: 1.6 s
Wall time: 1.19 s


In [53]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 73.22%


## noise 6 - sigma-0.2

In [55]:
# noise 6
noise6 = np.random.normal(0,0.2,x_test.shape)
x_test_noise6 = x_test_norm + noise6
x_test_noise6_reshape = x_test_noise6.reshape(-1,28,28,1)

In [56]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise6_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0454323291778564
CPU times: user 1.28 s, sys: 99.1 ms, total: 1.38 s
Wall time: 1.05 s


In [57]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 47.839999999999996%


# 1-2. FGSM eps=0.2

## 1-1) FGSM_origin
### 원본 이미지 x_test

In [58]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_reshaped,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1974897384643555
CPU times: user 1.55 s, sys: 117 ms, total: 1.67 s
Wall time: 1.2 s


In [59]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 59.089999999999996%


## 1-2) FGSM_noise
### noise image -> x_test_noise

In [60]:
# noise 
noise = np.random.normal(0,0.01,x_test.shape)
x_test_noise1 = x_test_norm + noise
x_test_noise1_reshape = x_test_noise1.reshape(-1,28,28,1)

In [61]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise1_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1353540420532227
CPU times: user 1.36 s, sys: 113 ms, total: 1.48 s
Wall time: 1.14 s


In [62]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 59.0%


## noise 2 - sigma-0.03

In [63]:
# noise 2
noise2 = np.random.normal(0,0.03,x_test.shape)
x_test_noise2 = x_test_norm + noise2
x_test_noise2_reshape = x_test_noise2.reshape(-1,28,28,1)

In [64]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise2_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0824291706085205
CPU times: user 1.33 s, sys: 110 ms, total: 1.44 s
Wall time: 1.08 s


In [65]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 56.75%


## noise 3 - sigma-0.05

In [66]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [67]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise3_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.2144227027893066
CPU times: user 1.51 s, sys: 133 ms, total: 1.65 s
Wall time: 1.21 s


In [68]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 53.89000000000001%


## noise 4 - sigma-0.07

In [69]:
# noise 4
noise4 = np.random.normal(0,0.07,x_test.shape)
x_test_noise4 = x_test_norm + noise4
x_test_noise4_reshape = x_test_noise4.reshape(-1,28,28,1)

In [70]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise4_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.3031105995178223
CPU times: user 1.67 s, sys: 121 ms, total: 1.79 s
Wall time: 1.3 s


In [71]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 50.470000000000006%


## noise 5 - sigma-0.1

In [72]:
# noise 5
noise5 = np.random.normal(0,0.1,x_test.shape)
x_test_noise5 = x_test_norm + noise5
x_test_noise5_reshape = x_test_noise5.reshape(-1,28,28,1)

In [73]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise5_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.049828290939331
CPU times: user 1.27 s, sys: 127 ms, total: 1.4 s
Wall time: 1.05 s


In [74]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 45.79%


## noise 6 - sigma-0.2

In [75]:
# noise 6
noise6 = np.random.normal(0,0.2,x_test.shape)
x_test_noise6 = x_test_norm + noise6
x_test_noise6_reshape = x_test_noise6.reshape(-1,28,28,1)

In [76]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise6_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0588874816894531
CPU times: user 1.32 s, sys: 93.3 ms, total: 1.41 s
Wall time: 1.06 s


In [77]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 24.2%


# 1-3. FGSM eps=0.3

## 1-1) FGSM_origin
### 원본 이미지 x_test

In [78]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_reshaped,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0228149890899658
CPU times: user 1.29 s, sys: 95.1 ms, total: 1.39 s
Wall time: 1.02 s


In [79]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 37.92%


## 1-2) FGSM_noise
### noise image -> x_test_noise

In [80]:
# noise 
noise = np.random.normal(0,0.01,x_test.shape)
x_test_noise1 = x_test_norm + noise
x_test_noise1_reshape = x_test_noise1.reshape(-1,28,28,1)

In [81]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise1_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0267248153686523
CPU times: user 1.28 s, sys: 87.7 ms, total: 1.37 s
Wall time: 1.03 s


In [82]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 37.49%


## noise 2 - sigma-0.03

In [83]:
# noise 2
noise2 = np.random.normal(0,0.03,x_test.shape)
x_test_noise2 = x_test_norm + noise2
x_test_noise2_reshape = x_test_noise2.reshape(-1,28,28,1)

In [84]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise2_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1029441356658936
CPU times: user 1.39 s, sys: 102 ms, total: 1.5 s
Wall time: 1.1 s


In [85]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 35.24%


## noise 3 - sigma-0.05

In [86]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [87]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise3_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1114296913146973
CPU times: user 1.41 s, sys: 91.4 ms, total: 1.5 s
Wall time: 1.11 s


In [88]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 33.29%


## noise 4 - sigma-0.07

In [89]:
# noise 4
noise4 = np.random.normal(0,0.07,x_test.shape)
x_test_noise4 = x_test_norm + noise4
x_test_noise4_reshape = x_test_noise4.reshape(-1,28,28,1)

In [90]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise4_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1905481815338135
CPU times: user 1.48 s, sys: 148 ms, total: 1.63 s
Wall time: 1.19 s


In [91]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 30.84%


## noise 5 - sigma-0.1

In [92]:
# noise 5
noise5 = np.random.normal(0,0.1,x_test.shape)
x_test_noise5 = x_test_norm + noise5
x_test_noise5_reshape = x_test_noise5.reshape(-1,28,28,1)

In [93]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise5_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1356945037841797
CPU times: user 1.43 s, sys: 98.2 ms, total: 1.53 s
Wall time: 1.14 s


In [94]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 27.24%


## noise 6 - sigma-0.2

In [95]:
# noise 6
noise6 = np.random.normal(0,0.2,x_test.shape)
x_test_noise6 = x_test_norm + noise6
x_test_noise6_reshape = x_test_noise6.reshape(-1,28,28,1)

In [96]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise6_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1915056705474854
CPU times: user 1.5 s, sys: 118 ms, total: 1.62 s
Wall time: 1.19 s


In [97]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 15.509999999999998%
